<a href="https://colab.research.google.com/github/opencae/hpc-training-gpu-2023/blob/main/1_hpc_tc_cavity_benchmark_cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「Google ColaboratoryのCPU・GPUを用いた3次元キャビティ・ベンチマークテスト実行」(演習) 「その1 CPU編」

オープンCAE学会 OpenFOAM GPUトレーニング2023 実行委員会 https://opencae.gitlab.io/web/page/training/opencae_hpc_training_gpu_2023/

## セルの実行と移動
セルのコードを実行するには、セルをクリックして選択し、コードの左側にある実行ボタンをクリックします。

実行ボタンにマウスカーソルが乗ると表示されるように、「Commandキー+Enterキー」または「Ctrlキー+Enterキー」のキーボードショートカットによっても実行できます。

さらに、「Shiftキー+Enterキー」のキーボードショートカットによって、 セルのコードを実行(テキストの場合はフォーカス)した後に、次のセルに移動しますので、「Shiftキー+Enterキー」によって、効率的にノートブックの閲覧と実行を進めることができます。

## イントロダクション


### GPUシステムへの移行
 東京大学情報基盤センターと筑波大学計算科学研究センターが共同で運営する最先端共同HPC基盤施設(JCAHPC)が次期に導入するスパコンOFP-IIは、NVIDIA H100及びその後継機種のGPUを中心としたシステムとなります。

* GPU移行 GPU移行に関するポータルサイト https://jcahpc.github.io/gpu_porting/

また、GPUを用いることが多いAI分野やLLM分野での研究の急速な発展により、JCAHPCに限らず、日本の他のスパコンにおいても、アクセラレータとしてGPUを用いるシステムの導入が続いています。

* HPCI 令和6年度課題募集におけるハードウェア・ソフトウェア資源一覧 https://www.hpci-office.jp/using_hpci/hardware_software_resource/2024

今後数年のスパンでは、スパコンなどの大規模計算機がGPU中心のシステムに移行することが予想されますので、大規模計算機を用いてOpenFOAMの解析を効率的に行うには、GPU移行を早目に検討する必要があります。


### OpenFOAMでのGPU利用


#### OpenFOAMのGPU移植版RapidCFD
OpenFOAMのGPU移植版としては、昔からRapidCFDがありましたが、開発はほとんど止まっているほか、かなり古いバージョンであるOpenFOAM-2.3.1の部分移植なので、移植されていない機能はもちろんのこと、それ以降のOpenFOAMの新しい機能が使えないという致命的な問題があります。

* 山岸孝輝: OpenFOAMのメニーコア・GPUへの対応に向けた取り組みの紹介オープンCAEシンポジウム2017 https://www.opencae.or.jp/wp-content/uploads/2018/02/sym2017_C23.pdf


#### 標準のOpenFOAMのexternal-solverモジュール
 一方、最近の標準版OpenFOAMには、PETScという線形ソルバのバライブラリを用いるための外部モジュールであるexternal-solverが用意されているため、PETScの機能を活かして、CPUだけでなく、GPUも併用した線形ソルバの実行が行えます。

external-solverは線形ソルバのみ外部モジュールを用いて別途計算を行うプラグイン的手法であることから、RapidCFDと異なり、標準版OpenFOAMの機能がそのまま使えるのが大きな利点となりますが、その手法の性質上、線形ソルバ部のみしかGPU化ができず、CPUとGPUのメモリ間のデータ転送も自ずと増えるので、高速化の余地が限られるのが欠点となります。

* external-solver
 https://develop.openfoam.com/modules/external-solver
* S. Bnà, I. Spisso, M. Olesen, G. Rossi PETSc4FOAM: A Library to plug-in PETSc into the OpenFOAM
Framework https://prace-ri.eu/wp-content/uploads/WP294-PETSc4FOAM-A-Library-to-plug-in-PETSc-into-the-OpenFOAM-Framework.pdf

#### GPU併用の試行
以上のように、OpenFOAMの解析においてGPUを使用する手法には欠点や限界も多い現状ですが、今後GPUをアクセラレータとするシステムが増えることを鑑みると、CPUのみを使用した解析とGPUを併用した解析の計算時間や費用効果を比較しておき、GPU併用によりこれらの各種計算効率の向上が見込める問題については、GPU併用計算への移行を検討しても良い時期にきていると言えます。


### OpenFOAMベンチマークテスト
CPUとGPU併用での各種計算効率を比較する解析対象は、最終的には自分が解析したい問題にする必要があるものの、最初の検討としては、計算条件や既往の比較結果が入手可能である、よく知られたベンチマークテストを検討するのが望ましいため、本演習でもOpenFOAM向けのベンチマークテストとして最も有名なHigh Performance ComputingTechinical Committeeの3D Lid-driven cavity benchmarkを対象とします。


### GPUが使用できる同一の演習計算環境
 本演習ではCPUとGPUの計算効率の比較を目的とするため、CPUだけでなく、GPUのハードウェア・リソースが必要となりますが、参加者各自にGPUを用意して頂くのは難しいです。

さらに、GPUを使うための各種ソフトウェアをインストールしたり、コンパイルする必要があるので、参加者の計算環境が同一でないと、トラブルが生じる可能性が高くなります

そこで、本演習では、無償のプランでもあってもGPUをバックエンドのハードウェア・アクセラレータとして用いることが可能であり、かつ計算環境が統一できるGoogle Colaboratory(Colab)を用います。


### 本演習の内容
演習の「その1 CPU編」ではColabのCPUのみを用いて、 HPC TC委員会のベンチマークテストを実行します。

CPUのみを用いる場合、OpenFOAM-v2306の非定常層流解析ソルバicoFoamを用い、PCGやPBiCGなど、CPUのみを用いる線形ソルバを指定します。

また、GPU上でも実行できる線形ソルバのライブラリであるPETScと、それをOpenFOAMから使用できるようする外部モジュールであるexternal-solverをソースコードからビルドします。

なお、これらのビルドにはGPUは必要がないので、CPU編で行なっておき、ビルドしたライブラリをGoogleドライブにコピーしておくことにより、GPU編ですぐに使用できるようにします。

演習の「その2 GPU編」ではハードウェア・アクセラレータとしてGPUを用いて解析するために、GPU向けに移植されたRapicCFDの解析ソルバを用いたケースを解析します。

また、OpenFOAM-v2306の非定常層流解析ソルバicoFoamとPETScライブラリを用い、GPU上でCG法線形ソルバを実行するケースも解析します。

PETScのCG法線形ソルバをGPU上で走らせる場合、前処理等の条件によって計算時間が大きく変化しますので、最後のカスタマイズ演習では、PETScの計算条件カスタマイズを行ないます。


### 本演習の目次
本演習の内容はColabのノートブックに記述されているので、詳しい目次については、左の「目次」メニューから参照してください。

## High Performance ComputingTechinical Committeeの3次元キャビティ解析ベンチマークテストとは

High Performance ComputingTechinical Committee(HPC TC)とは、OpenFOAMのTechnical Committees内に設置されたOpenFOAMのHigh Performance Computing(HPC)向けのベンチマークテスト制定や性能向上を目的とした委員会です。

* OpenFOAM® Technical Committees https://www.openfoam.com/governance/technical-committees
* High Performance Computing (HPC) Technical Committee https://wiki.openfoam.com/High_Performance_Computing_(HPC)_Technical_Committee

また、本委員会が制定するベンチマークテストのうち、最も代表的なものが3次元キャビティ解析ベンチマークテスト(3D Lid-driven cavity benchmark)です。

* Repository for the High-Performance Computing Technical Committee https://develop.openfoam.com/committees/hpc/-/wikis/home
  * 3-D Lid Driven cavity flow https://develop.openfoam.com/committees/hpc/-/wikis/home#3-d-lid-driven-cavity-flow
  * 3D Lid-driven cavity benchmark https://develop.openfoam.com/committees/hpc/-/wikis/home#3d-lid-driven-cavity-benchmark

ちなみに、「富岳」でのOpenFOAMの性能測定にも、このベンチマークテストが使われたことがあります。

* スーパーコンピュータ「富岳」 | HPCI https://www.hpci-office.jp/for_users/appli_software/appli_openfoam/openfoam_r-ccs_riken-2
  * OpenFOAMの性能情報 http://www.hpci-office.jp/documents/appli_software/Fugaku_OpenFOAM_performance.pdf

また、HPC TCは数値計算ライブラリPETScなど、CPU以外にもGPUも使用できる外部ソルバーを任意のOpenFOAMソルバーから使用できるようにするexternal-soverモジュール(petscFoam, PETSc4FOAM)の開発にコミットしています。

* external-solver https://develop.openfoam.com/modules/external-solver
* S. Bnà, I. Spisso, M. Olesen, G. Rossi PETSc4FOAM: A Library to plug-in PETSc into the OpenFOAM Framework PRACE White paper https://prace-ri.eu/wp-content/uploads/WP294-PETSc4FOAM-A-Library-to-plug-in-PETSc-into-the-OpenFOAM-Framework.pdf

## Google Colaboratoryとは

Google Colaboratory(Colab)とは、以下のWebページに記載の通り、*「特別な設定なしでご利用いただけるホスト型の Jupyter Notebook サービス」*です。

* Colaboratory よくある質問 https://research.google.com/colaboratory/faq.html?hl=ja

上記のWebページにも記載されていますが、Colabには様々なリソース制限があります。
予め以下のWebページも参照ください。

* Google Colabの制限と対策 https://note.com/npaka/n/n1aa6f8c973d0

Colabではノートブックの最大実行時間を過ぎると、ファイルが初期化されるので、コンパイル結果、ベンチマークテストのカスタマイズ設定、実行結果など、生成に手間や時間がかかるファイルは、ここでは参加者各自のGoogleドライブに退避しておきます。


## Google ドライブをマウントする


以下にGoogle ドライブをマウントする方法を示します。
Google のアカウントが必要になりますので、予め作成しておいてください。

Google ドライブをマウントには、下記のセルのコードを実行してください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

既にマウント済みでない場合には、下記のようなメッセージが出ると思いますので、**Googleドライブに接続**を選択します。

---

**このノートブックに Google ドライブのファイルへのアクセスを許可しますか？**

このノートブックは Google ドライブ ファイルへのアクセスをリクエストしています。Google ドライブへのアクセスを許可すると、ノートブックで実行されたコードに対し、Google ドライブ内のファイルの変更を許可することになります。このアクセスを許可する前に、ノートブック コードをご確認ください。

スキップ  **Googleドライブに接続**

---


その後、アカウントの選択を求められた場合には、使用する**Googleアカウントを選択**します。

さらに、Googleアカウントへのアクセスをリクエストされた場合には、適宜**許可**します。

上記のようなgoogle の認証手順を踏んでGoogleドライブのマウントに成功すると、'Mounted at /content/drive'と出力され、Googleドライブが /content/drive/MyDrive にマウントされますので、以下を実行して、マウントされたディレクトリを表示してください。

In [ ]:
%ls /content/drive

'MyDrive/'と表示されれば問題ありません。
（'MyDriver/' だけではなく、 'Othercomputers/' などもあわせて表示される可能性があります。）

## システムコマンドおよびマジックコマンドの実行
### システムコマンドの実行
Google Colabはホスト型の Jupyter Notebook サービスであり、IPythonをバックエンドで使っています。
IPythonでは!pwdのように先頭に!をつけることでシステムコマンドを実行できます。

In [ ]:
!pwd

ただし、カレントディレクトリを変更するシステムコマンドはcdですが、これを!cdとして実行しても、反映されません。

これは以下の2つのセルを実行して確認することができます。

In [ ]:
!cd /content/drive/MyDrive

In [ ]:
!pwd

!をつけたシステムコマンドが実行されたシェルは即座に廃棄されるため、!cdに限らずシェルの状態・設定を変更するようなコマンドを!で実行しても反映されません。

従って、カレントディレクトリを変更する場合には、以下のマジックコマンド版のcdを用います。

### マジックコマンド

IPythonでは%から始まるマジックコマンドが使えます。

#### マジックコマンド一覧表示
%lsmagicで現在利用可能なマジックコマンド一覧を表示できます。


In [ ]:
%lsmagic

なお、'Automagic is ON'の場合、前に%を付けなくても、マジックコマンドを実行することが可能です。

Googleドライブのマウント確認のノートプック( https://colab.research.google.com/github/opencae/hpc-training-gpu-2023/blob/main/mount_google_drive.ipynb )において、'ls /content/drive'と、lsの前に%が付いていないのに、 lsが実行できたのは、デフォルトで'Automagic is ON'となっているからです。

なお、AutomagicのONとOFFは、以下のマジックコマンドにより変更できます。

In [ ]:
%automagic

In [ ]:
%automagic

以降では、念のためAutomagicの設定に依存せずにマジックコマンドが実行できるよう%を省略しません。

### マジックコマンドによるカレントディレクトリの変更
%cd によってカレントディレクトリが変更できます。

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!pwd

## リソースの確認

### ハードウェア
最近のLinuxシステムでは、カーネルの現在の状態を表す特殊なファイルの階層であるprocファイルシステム上の仮想ファイルを読むことで、様々な状態を知ることができます。

#### CPUの状態確認
仮想ファイル/proc/cpuinfoの中身を読み出すことによって、CPUの状態確認ができます。

In [ ]:
!cat /proc/cpuinfo

#### 物理CPUの数
物理CPU数は、/proc/cpuinfoにおけるphysical.idのユニークな数です。

In [ ]:
!grep physical.id /proc/cpuinfo | sort -u | wc -l

#### 論理プロセッサーの数

論理プロセッサー数は、/proc/cpuinfoにおけるprocessor数です。

In [ ]:
!grep processor /proc/cpuinfo | wc -l

#### lscpuによる確認
`lscpu`コマンドがインストールされている場合には、本コマンドによってもCPUの状態が確認できます。

In [ ]:
!lscpu

#### メモリの状態確認
仮想ファイル/proc/meminfo の中身を読み出すことによって、メモリの状態確認ができます。

In [ ]:
!cat /proc/meminfo

なお、「メニュー→ランタイム→リソースの表示」により、リソースが表示され、システムRAMの使用容量/最大使用可能容量が表示されます。

### カーネル・ディストリビューション・ソフトウェア
カーネルやディストリビューション、および、インストールされているソフトウェアのバージョン等を確認しておきます。

#### カーネル
unameコマンドにより、現在稼働中のカーネルについての情報を得ることができます。

* Ubunti Manpage: uname https://manpages.ubuntu.com/manpages/jammy/ja/man2/uname.2.html

In [ ]:
!uname -a

カーネルはLinux 5.15.109+です(64bit版x86バイナリ)。

なお、仮想ファイル/proc/versionからもカーネルのバージョンがわかります。

In [ ]:
!cat /proc/version

#### ディストリビューション

ディストリビューション名やバージョンを表示する方法は、ディストリビューションによって異なりますが、通常 /etc/issue 、または、/etc/*-release の内容から知ることができますますので、これらのファイルの中身をcatやtailなどで表示します。

さらに、以下のようにtailコマンドの引数に複数のファイルを指定すると、ファイル名が表示されるので便利です。

また、末尾から10行のみを表示するのではなく、先頭行から全てを表示するには、'-n +1'(-n +開始行)のオプションを付けます。

In [ ]:
!tail -n +1 /etc/issue /etc/*-release

上記により、ディストリビューションはUbuntu系とわかります。

#### 標準でインストールされているGccコンパイラのバージョン

In [ ]:
!gcc -v

## OpenFOAM-v2306のインストール
ディストリビューションはUbutuですから、OpenFOAM-v2306のコンパイル済みのUbuntu向けパッケージをインストールします。

* Precompiled packages (Debian, Ubuntu) https://develop.openfoam.com/Development/openfoam/-/wikis/precompiled/debian

### OpenFOAMレポジトリの追加

In [ ]:
!curl https://dl.openfoam.com/add-debian-repo.sh | bash

### OepnFOAM-v2306の最小限パッケージのインストール
最小限のパッケージであるopenfoam2306をインストールすれば、HPC TCベンチマークを標準のOpenFOAMを用いてCPU上で実行可能ですので、openfoam2306をインストールします。

* precompiled packages - Sub-packages https://develop.openfoam.com/Development/openfoam/-/wikis/precompiled#sub-packages

In [ ]:
!apt-get install openfoam2306

## High-Performance Computing Technical Committeeのレポジトリのクローン

Googleドライブ上に本トレーニング用のディレクトリhpc-training-gpu-2023を作成し、そこにHigh-Performance Computing Technical Committee(HPC TC委員会)のレポジトリをクローンします。

なお、修正した設定ファイルやCPU上で生成した格子ファイル、CPUでの解析ログなどを、GPUをはハードウェア・アクセラレータとする別のランタイムでも共有して使うため、本レポジトリをGoogleドライブ上に置いています。

In [ ]:
%mkdir -p /content/drive/MyDrive/hpc-training-gpu-2023
%cd /content/drive/MyDrive/hpc-training-gpu-2023
!git clone https://develop.openfoam.com/committees/hpc.git

## CPUを用いたキャビティベンチマークケースSの実行(ケースS-cpu)
ケースSは、立方体状のキャビティ流れの解析領域を100x100x100=1,000,0000(1M)の格子に分割したケースです。

### ケースディレクトリへの移動

In [ ]:
%cd /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/S

### ケースファイル修正
HPC TC委員会のベンチマークテストはOpenFOAM-v2006以降の新しいバージョン用であり、OpenFOAM-2.3.1をGPU向けに移植したRapicCFDでは実行できないので、ここではRapidCFDでも実行できるように設定を変更します。

また、短時間で実行できるように、2ステップのみ実行するように変更します。

さらに、同じディレクトリ上で、線形ソルバの設定を変えた様々なケースを実行するため、線形ソルバの設定ファイル`system/fvSolution`を`system/fvSolution.ケース名`として置いておき、解析ソルバの実行時に、`system/fvSolution.ケース名`を本来の設定ファイル`system/fvSolution`にコピーすることにします。

なお、本ランタイムの再接続時など、既に設定が変更されている場合に備え、`git checkout`でオリジナルのファイルをチェックアウトしてから変更します。

In [ ]:
!case=S-cpu;\
git checkout system/controlDict system/fvSchemes constant/transportProperties;\
sed -i -e 's|^\( *#includeFunc\)|//\1|' -e 's|\(endTime \).*|\1 0.002;|' system/controlDict;\
echo 'fluxRequired {default no;p;}' >> system/fvSchemes;\
sed -i s/'^\(nu\s*\[\)'/'nu \1'/ constant/transportProperties;\
sed s/PBiCGStab/PBiCG/ system/fvSolution.fixedNORM > system/fvSolution.$case

変更したファイルの内容を確認します。

In [ ]:
!case=S-cpu;\
tail -n+1 constant/transportProperties system/{controlDict,fvSchemes,fvSolution.$case}

### 格子生成

`blockMesh`アプリケーションを用いて格子を生成します。

OpenFOAMのアプリケーションを実行するには、通常OpenFOAMの環境設定が必要ですが、 !をつけたシステムコマンドが実行されたシェルは即座に廃棄されるため、OpenFOAMの環境設定を行なった同一シェル上で、アプリケーションを実行します。

さらに、ここでは念のため`blockMesh`の前に`time`コマンドを付けて、`blockMesh`コマンドの実行のリアルタイム時間、ユーザー時間、システム時間を表示させています。

* Man page of TIME https://linuxjm.osdn.jp/html/LDP_man-pages/man1/time.1.html

In [ ]:
!case=S-cpu;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
time blockMesh 2>&1 | tee log.blockMesh.$case

格子数は、nCellsにある通り1000000(100x100x100, 1M)です。

### CPUでのOpenFOAM v2306版ソルバ実行
`system/fvSolution.ケース名`を本来の設定ファイル`system/fvSolution`にコピーしてから解析ソルバを実行します。

In [ ]:
!case=S-cpu;\
cp system/fvSolution.$case system/fvSolution;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
time icoFoam 2>&1 | tee log.icoFoam.$case

### 毎ステップの実行時間

ソルバのログには、ExectutionTimeで始まる行に毎ステップ毎の実行時間が出力されているので、これらを抜き出して出力します。

In [ ]:
!case=S-cpu;\
grep '^ExecutionTime' log.icoFoam.$case

### 1ステップの実行時間

2ステップ目と1ステップ目のExectutionTimeの時間差より1ステップの実行時間を算出します。



In [ ]:
!case=S-cpu;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Execution time for one timestep : ",t[1]-t[0]}' log.icoFoam.$case

## CPUを用いたキャビティベンチマークケースMの実行(ケースM-cpu)
ケースMは、立方体状のキャビティ流れの解析領域を200x200x200=8,000,0000(8M)の格子に分割したケースであり、ケースSの8倍の格子数になっています。


### ケースディレクトリへの移動

In [ ]:
%cd /content/drive/MyDrive/hpc-training-gpu-2023/hpc/Lid_driven_cavity-3d/M

### ケースファイル修正
ケースSと同様にケースファイルを修正します。

In [ ]:
!case=M-cpu;\
git checkout system/controlDict system/fvSchemes constant/transportProperties;\
sed -i -e 's|^\( *#includeFunc\)|//\1|' -e 's|\(endTime \).*|\1 0.0005;|' system/controlDict;\
echo 'fluxRequired {default no;p;}' >> system/fvSchemes;\
sed -i s/'^\(nu\s*\[\)'/'nu \1'/ constant/transportProperties;\
sed s/PBiCGStab/PBiCG/ system/fvSolution.fixedNORM > system/fvSolution.$case


変更したファイルの内容を確認します。

In [ ]:
!case=M-cpu;\
tail -n+1 constant/transportProperties system/{controlDict,fvSchemes,fvSolution.$case}

### 格子生成

In [ ]:
!case=M-cpu;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
time blockMesh 2>&1 | tee log.blockMesh.$case

格子数はnCellsに示す通り、8000000(200x200x200, 8M)です

### CPUでのOpenFOAM v2306版ソルバ実行

格子数が大きく、CPU1コアでの実行なので、**完了までには十数分間かかります(割当てられているCPUによって時間は異なります)**。

なお、「メニュー→ランタイム→リソースを表示」により、システムRAMの容量や使用量がモニターできますが、本ケースでは、システムRAMの12.7GBのうち7GB以上を使用していますので、格子数が本ケースの8倍であるケースXLはメモリ不足で解析できないことがわかります。

In [ ]:
!case=M-cpu;\
cp system/fvSolution.$case system/fvSolution;\
source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
time icoFoam 2>&1 | tee log.icoFoam.$case

### 毎ステップの実行時間

In [ ]:
!case=M-cpu;\
grep '^ExecutionTime' log.icoFoam.$case

### 1ステップの実行時間

In [ ]:
!case=M-cpu;\
awk 'BEGIN {n=0} /ExecutionTime/ {t[n]=$3;n++} END {print "Execution time for one timestep : ",t[1]-t[0]}' log.icoFoam.$case

## NVIDIAのGPU上で線形ソルバを実行できるPETScライブラリおよびexternal-solverモジュールのビルド
数値計算ライブラリPETSc、および、それをOpenFOAMで用いるためのexternal-solverモジュールを用いることで、標準のOpenFOAMであっても、PETScを通じて線形ソルバをGPU上で実行することが可能となります。

* PETSc https://petsc.org/
* external-solver https://develop.openfoam.com/modules/external-solver

ただし、Ubutu用パッケージにはexternal-solver等のモジュールが含まれていないので、external-solverモジュールを別途コンパイルする必要があります。

モジュール等をソースからコンパイルするには、openfoam2306といった最小限のパッケージだけでなく、 ソースコードやwmakeといったコンパイル設定が含まれたopenfoam2306-sourceパッケージ、および、ビルドツールが含まれたopenfoam2306-toolsパッケージをインストールします。

* precompiled packages - Sub-packages https://develop.openfoam.com/Development/openfoam/-/wikis/precompiled#sub-packages

なお、開発環境が全て含まれたopenfoam2306-devパッケージ、もしくは、チュートリアルなど全てを含んだopenfoam2306-defaultパッケージをインストールしても、PETScやexternal-solverモジュールはコンパイル可能ですが、これらのパッケージは容量が大きいのでインストールに時間がかかります。


### OpenFOAMのソースコード・ビルドツールのインストール

In [ ]:
!apt-get install openfoam2306-source openfoam2306-tools

### PETScライブラリのビルド

数値計算ライブラリPETScには、OpenFOAM同様にUbutu向けパッケージpetsc-devもありますが、これはNVIDIAのGPU上で線形ソルバを実行できるようにビルドされていないので、ここではThirdPartyパッケージに含まれる`makePETSC`コマンドを用いて、ソースコードからビルドします。

* PETSc https://petsc.org/

OpenFOAMのトップディレクトリに移動します。

In [ ]:
%cd /usr/lib/openfoam/openfoam2306

ThirdPartyパッケージは、OpenFOAM-v2306用のアカーイブファイルも配布されていますが、PETSc以下の多くのライブラリのソースコードを含んでおり容量も大きいので、ここでは、ソースコードを含まないThirdParty-commonレポジトリをクローンします。

In [ ]:
!git clone https://develop.openfoam.com/Development/ThirdParty-common.git

UbuntuのパッケージではダミーのファイルとなっているThirdPartyを消去します。

In [ ]:
!rm ThirdParty

Ubutuのopenfoam2306パッケージでは、ThirdPartyのディレクトリが /usr/lib/openfoam/openfoam2306/ThirdParty と設定されているので、 ThirdParty-commonディレクトリをThirdPartyに変更します。

In [ ]:
!mv ThirdParty-common ThirdParty/

ThirdPartyディレクトリに移動します。

In [ ]:
%cd ThirdParty/

ThirdPartyのソースコードはsoucesディレクトリ下に置くことになっているので、 soucesディレクトリを作成した後、PETScのソースコードのアーカイブ・ファイルをダウンロードして、soucesディレクトリ下に展開します。

In [ ]:
!mkdir sources;\
wget -O - https://ftp.mcs.anl.gov/pub/petsc/release-snapshots/petsc-lite-3.19.2.tar.gz | tar zx -C sources

PETScライブラリをCUDA有効の設定でビルドしますが、**本ビルドには十数分間かかります(割当てられているCPUによって時間は異なります)**。


In [ ]:
!source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
WM_NCOMPPROCS=2 ./makePETSC -no-hypre -- --with-cuda 2>&1 | tee log.makePETSC

なお、3次元キャビティベンチマークのケースSやMにおいて、圧力に関する線形ソルバの前処理はDIC(Diagonal-based Incomplete Cholesky)分解法であり、AMG(Algebraic Multigrid)法を用いておりません。

また、今回は演習時間の関係で、Multigrid Methodsを含むHYPREライブラリやKokkosライブラリのビルドは行わないので、PETScもHYPER無効でビルドを行なっています。

* HYPRE: Scalable Linear Solvers and Multigrid Methods https://computing.llnl.gov/projects/hypre-scalable-linear-solvers-multigrid-methods
* PCHYPER https://petsc.org/main/manualpages/PC/PCHYPRE/
* Hypre / BoomerAMG https://mooseframework.inl.gov/releases/moose/2021-05-18/application_development/hypre.html
* Kokkos https://github.com/kokkos/kokkos

HYPREやKokkosを有効にする場合には、以下のサイトなどを参照してください。

* OpenFOAM v2206 を Ubuntu 20.04 LTS にインストールメモ https://ss1.xrea.com/penguinitis.g1.xrea.com/study/OpenFOAM/install_memo/OpenFOAM_v2206-Ubuntu20_04_LTS_PETSc.html

ちなみに、`WM_NCOMPPROCS=2`を設定して2並列ビルドの設定をしているのにもかかわらず、 ビルドログの途中で
 `gmake[3]: warning: jobserver unavailable: using -j1.  Add '+' to parent make rule.`といったワーニングが出力されていますが、PETSc-3.19.5といったより新しいバージョンではこのワーニングは出現しません。

PETScのバージョンを変更するには、 /usr/lib/openfoam/openfoam2306/etc/config.sh/petsc における `petsc_version` を変更すれば良いのですが、ここではデフォルトのままにしておきます。

### external-solverモジュールのビルド

Ubuntuのopenfoam2306パッケージには、本来external-solver等のパッケージが置かれるmodulesディレクトリが含まれていないので、これを作成します。

In [ ]:
!mkdir /usr/lib/openfoam/openfoam2306/modules

modulesディレクトリに移動します。

In [ ]:
%cd /usr/lib/openfoam/openfoam2306/modules

external-solverのソースコードをgitでcloneします。

In [ ]:
!git clone https://develop.openfoam.com/modules/external-solver.git

external-solverディレクトリに移動します。

In [ ]:
%cd external-solver

external-solverモジュールをビルドします。
なお、`Allwmake`に`-j`オプションを付けると、全論理コア数(ここでは2)の並列ビルドになります。
また、ライブラリや実行バイナリのインストール先を、`-prefix`オプションを用いてopenfoam2306のシステムと同じ場所を指定しています。

In [ ]:
!source /usr/lib/openfoam/openfoam2306/etc/bashrc;\
./Allwmake -j -prefix=/usr/lib/openfoam/openfoam2306/platforms/linux64GccDPInt32Opt 2>&1 | tee log.Allwmake

上記のログに`Compiling enabled on 2 cores`と記載されていることから、2並列でビルドされていることがわかります。

また、PETScのライブラリを有効にするには、`eval $(foamEtcFile -sh -config petsc -- -force)`を実行すれば良いことがわかります。

### PETScライブラリおよびexternal-solverモジュールのライブラリのGoogleドライブへの保存

OpenFOAMのトップディレクトリに移動します。

In [ ]:
%cd /usr/lib/openfoam/openfoam2306

ビルドしたPETScライブラリとexternal-solverモジュールのライブラリのアーカイブファイルをGoogleドライブに保存します。

In [ ]:
!tar Jcf /content/drive/MyDrive/hpc-training-gpu-2023/petsc.tar.xz ThirdParty/platforms platforms/linux64GccDPInt32Opt/lib/libpetscFoam.so

ノートブックのファイルが初期化された場合には、OpenFOAMのパッケージは再インストールする必要がありますが、Googleドライブに退避された上記のアーカイブファイルを解凍することで、PETScライブラリやexternal-solverモジュールを再度ビルドすることなく、すぐに使用することができます。

なお、Googleドライブ上に直接OpenFOAMのトップディレクトリを置いて、その下でPETScライブラリやexternal-solverモジュールをビルドすることも可能であり、この手法ではOpenFOAMのパッケージの再インストールやアーカイブファイルの解凍の手間がかかりません。

ただし、Googleドライブに再接続した際にファイルの実行フラグが無くなるので、実行フラグが必要なファイルの実行フラグを再設定する必要が生じます。

* Mounting google drive with permition to execut binaries #3162 https://github.com/googlecolab/colabtools/issues/3162

また、Googleドライブ内のファイルに頻繁にアクセスすることになるので、GoogleドライブのAPIコールの制限にかかる可能性が高くなります。

従って、ここではGoogleドライブ上にファイルの属性が保存されるアーカイブファイルを置いておき、ノートブックのファイルが初期化された場合や、ハードウェア・アクセラレータとしてGPUを違うなど異なるランタイムタイプを使用する場合には、必要なOpenFOAMのパッケージを再インストールした上で、必要なアーカイブファイルを解凍することにします。



**以上で、「その1 CPU編」のノートブックは終了です。**